### **Data Cleaning and Exploratory Analysis**

#### **Import libraries**

In [2]:
# Import the custom function 'gera_dados' from the local module 'gerar_dados_aleatorios'
from modules.generae_random_sales_data import GenerateData as gd

# Import date and time utilities for handling timestamps and time intervals
from datetime import datetime, timedelta

# Import data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Import data manipulation and numerical computation libraries
import pandas as pd
import numpy as np

# Import the 'random' module for generating random numbers
import random

#### **Create functions**

In [3]:
# Function to format numeric values into millions with a currency symbol (Brazilian Real)
# Example: 3500000 -> 'R$ 3.50 Mi'
def formatador_milhoes(y, pos):
    return f'R$ {y/1_000_000:.2f} Mi'

In [3]:
# Function to format percentage labels for charts (e.g., pie charts)
# It shows both the percentage and the corresponding absolute value in millions
# Example: 25%, R$ 2.50 Mi
def formatar_valores(pct, allvals):
    absolute = pct/100. * sum(allvals)
    return f'{pct:.1f}%\nR$ {absolute/1_000_000:.2f} Mi'

#### **Create a random dataset**

In [4]:
# Create a random dataFrame
df_sales = gd.generate_fake_data()


Starting the generation of 600 records...
Data generation completed successfully.



In [5]:
# Check the first lines of the DataFrame
df_sales.head()

,Order_ID,Order_Date,Product_Name,Category,Unit_Price,Quantity,Customer_ID,City,State
0,1000,2026-01-01 15:00:00,Vertical Mouse,Accessories,241.08,5,128,São Paulo,SP
1,1001,2026-01-01 02:00:00,Graphics Card,Hardware,4500.00,1,100,Belo Horizonte,MG
2,1002,2026-01-01 09:00:00,SSD 1TB,Hardware,600.00,2,122,Salvador,BA
3,1003,2026-01-01 22:00:00,Graphics Card,Hardware,4500.00,7,106,Fortaleza,CE
4,1004,2026-01-01 15:00:00,Gaming Laptop,Electronics,7500.00,5,109,Fortaleza,CE
